<a href="https://colab.research.google.com/github/pavi-ninjaac/Specrom_Analysis/blob/main/Internship_works/week3/text_classification_109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#get the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

#sklearn functions
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import LabelEncoder

#sklearn vectorization packages
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/Text_classification_news_article/rishika_csv_different_computer_with_iptc.csv',index_col=0)

shape = data.shape
columns = list(data.columns)

print("the shape of the dataset is--->",shape)
print('The features are-------------->',columns)

data.head(4)


the shape of the dataset is---> (10761, 8)
The features are--------------> ['position', 'query', 'url', 'title', 'snippet', 'combined_text', 'level1_labels', 'level2_labels']


,position,query,url,title,snippet,combined_text,level1_labels,level2_labels
0,1,"arts, culture, entertainment and media",https://www.bbc.com/news/entertainment_and_arts,Entertainment & Arts - BBC News,Get the latest BBC Entertainment and Arts news...,Entertainment & Arts - BBC News Get the latest...,"arts, culture, entertainment and media",NaN
1,2,"arts, culture, entertainment and media",https://www.champaigncounty.org/list/ql/arts-c...,"Arts, Culture & Entertainment | Champaign Coun...",Champaign County Chamber of Commerce - IL 303 ...,"Arts, Culture & Entertainment | Champaign Coun...","arts, culture, entertainment and media",NaN
2,3,"arts, culture, entertainment and media",https://www.bls.gov/oes/current/oes270000.htm,"Arts, Design, Entertainment, Sports, and Media...","27-0000 Arts, Design, Entertainment, Sports, a...","Arts, Design, Entertainment, Sports, and Media...","arts, culture, entertainment and media",NaN
3,4,"arts, culture, entertainment and media",https://www.smc.edu/academics/areas-of-interes...,"Arts, Media, & Entertainment - Santa Monica Co...",Let your imagination run wild and free — explo...,"Arts, Media, & Entertainment - Santa Monica Co...","arts, culture, entertainment and media",NaN


In [38]:
print("The nunber of category----------->",data['level1_labels'].nunique())
print("The nunber of second category---->",data['level2_labels'].nunique())

The nunber of category-----------> 17
The nunber of second category----> 109


In [39]:
print("The nunber of quieries---->",data['query'].nunique())

The nunber of quieries----> 1296


In [40]:
data['position'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [41]:
needed_columns = ['combined_text','query','level1_labels','level2_labels']
data = data[needed_columns].copy()
data.head(4)

,combined_text,query,level1_labels,level2_labels
0,Entertainment & Arts - BBC News Get the latest...,"arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
1,"Arts, Culture & Entertainment | Champaign Coun...","arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
2,"Arts, Design, Entertainment, Sports, and Media...","arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
3,"Arts, Media, & Entertainment - Santa Monica Co...","arts, culture, entertainment and media","arts, culture, entertainment and media",NaN


# Handling dublicate values

In [42]:
#delete the dublicate values
data.duplicated().sum() # count the total duplicate samples

362

In [43]:
data.drop_duplicates(keep='last',inplace=True)

In [44]:

#there can be dublicate of author names so check for the dublicate headline and short discription
data.duplicated(subset=[ 'combined_text']).sum()

751

In [45]:
data.drop_duplicates(subset='combined_text' , keep ='last' , inplace=True)

In [46]:
print("THe length of the datset after dublicate deletion------>",data.shape)

THe length of the datset after dublicate deletion------> (9648, 4)


# Handling null values

In [47]:
data.isnull().sum()

combined_text      1
query              0
level1_labels      0
level2_labels    133
dtype: int64

In [48]:
data.dropna(axis=0 ,inplace=True , how='any')

In [49]:
print("THe length of the datset after null values deletion------>",data.shape)

THe length of the datset after null values deletion------> (9514, 4)


In [50]:
data[data['combined_text'] == ' ']
data = data.reset_index()
data = data.drop('index',axis=1)
data.head(4)

,combined_text,query,level1_labels,level2_labels
0,A&E | Watch Full Episodes of Your Favorite Sho...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
1,A&E TV Shows | A&E Check out A&E's shows lineu...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
2,Entertainment & Arts - BBC News Get the latest...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
3,Arts & Entertainment | news-gazette.com · Clo...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment


# Cleaning the combined text

In [51]:
def preprocessor_text(text):
    if isinstance((text), (str)):
        text = re.sub('<[^>]*>', ' ', text)
        text = re.sub('[\W]+', ' ', text.lower())
        return text
    if isinstance((text), (list)):
        return_list = []
        for i in range(len(text)):
            temp_text = re.sub('<[^>]*>', '', text[i])
            temp_text = re.sub('[\W]+', '', temp_text.lower())
            return_list.append(temp_text)
        return(return_list)

data['combined_text'] = data['combined_text'].apply(preprocessor_text)

data.head(4)

,combined_text,query,level1_labels,level2_labels
0,a e watch full episodes of your favorite shows...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
1,a e tv shows a e check out a e s shows lineup ...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
2,entertainment arts bbc news get the latest bbc...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
3,arts entertainment news gazette com cloudy sno...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment


# Balance the dataset

### 'query' categories

- When doing the balancing , we have to delete the categorie, which are having very few sample. Here by i am deleting the categories which are having less than 6 sample. 
- after deletion, we have to resample the data by select 6 samples randomly from each categories. 

In [52]:
#copy the original data 
data = data.copy()
data_query = data[['combined_text','query']].copy()

In [53]:
#helper functions
def downsample(df , label):
  g = df.groupby(label , group_keys=False)
  balaced_df = pd.DataFrame(g.apply(lambda x : x.sample(g.size().min()))).reset_index(drop=True)
  return balaced_df


def balace_category(data , category_column , downsample_value):
  #to handle the categories easily, we have to change the dtype of the 'query' to category
  data[category_column] = data[category_column].astype('category')

  category = data[category_column].value_counts().index
  print("The shape before deletion------------------------------------------>",data.shape)
  print()
  print("The number of category in the query column------------------------->",len(category))

  #seperate the column which has to be delete
  need_to_be_deleted = category[(data[category_column].value_counts() < downsample_value)]
  print("The number of category need to be deleted in the query column------>",len(need_to_be_deleted))

  #delete the unwanted category
  data[category_column] = data[category_column].cat.remove_categories(need_to_be_deleted) #set nan for the categories need to be deletd and then use dropna to delete those things
  data.dropna(axis=0 , how='any',inplace=True) #delete the nan category

  print("The number of category after deletion------------------------------>",len(data[category_column].value_counts().index))
  print()
  print("The shape after deletion------------------------------------------->",data.shape)
  print()

  #down sample the query category to its minimum value
  data = downsample(data , category_column) #call the helper function
  #data[category_column].value_counts()

  return data

In [54]:
#to handle the categories easily, we have to change the dtype of the 'query' to category
data_query['query'] = data_query['query'].astype('category')





In [55]:
query_category = data_query['query'].value_counts().index
print("The shape before deletion------------------------------------------>",data_query.shape)
print()
print("The number of category in the query column------------------------->",len(query_category))

#seperate the column which has to be delete
need_to_be_deleted = query_category[(data_query['query'].value_counts()<6)]
print("The number of category need to be deleted in the query column------>",len(need_to_be_deleted))

#delete the unwanted category
data_query['query'] = data_query['query'].cat.remove_categories(need_to_be_deleted) #set nan for the categories need to be deletd and then use dropna to delete those things
data_query.dropna(axis=0 , how='any',inplace=True) #delete the nan category

print("The number of category after deletion------------------------------>",len(data_query['query'].value_counts().index))
print()
print("The shape after deletion------------------------------------------->",data_query.shape)
print()

The shape before deletion------------------------------------------> (9514, 2)

The number of category in the query column-------------------------> 1279
The number of category need to be deleted in the query column------> 252
The number of category after deletion------------------------------> 1027

The shape after deletion-------------------------------------------> (8446, 2)



In [56]:
#down sample the query category to its minimum value
data_query = downsample(data_query , 'query')
data_query['query'].value_counts()

zoology                            6
death and dying                    6
extradition                        6
extra lightweight (judo)           6
exports                            6
                                  ..
paediatrics                        6
oval track dog racing              6
outsourcing                        6
outdoor recreational activities    6
All Saints Day                     6
Name: query, Length: 1027, dtype: int64

In [58]:
d = 'query'
data[d]

0       arts and entertainment
1       arts and entertainment
2       arts and entertainment
3       arts and entertainment
4       arts and entertainment
                 ...          
9509           weather warning
9510           weather warning
9511           weather warning
9512           weather warning
9513           weather warning
Name: query, Length: 9514, dtype: object

In [ ]:
#downsample the data to its lowest value
g = data.groupby('query' , )

In [19]:
# checking the categori
df = pd.DataFrame({'x' : [1,2,3,4,5,6,7,8] , 'label':[1,2,1,2,3,1,2,3]})
df['label'] = df['label'].astype('category')
df.label.cat.categories

Int64Index([1, 2, 3], dtype='int64')

In [21]:
g = df.groupby('label' , group_keys = False)
balaced = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()))).reset_index(drop=True)
balaced

,x,label
0,6,1
1,3,1
2,2,2
3,7,2
4,5,3
5,8,3


In [23]:
g.size()

label
1    3
2    3
3    2
dtype: int64

In [63]:
df = df.label.cat.remove_categories(3)
df

0      1
1      2
2      1
3      2
4    NaN
5      1
6      2
7    NaN
Name: label, dtype: category
Categories (2, int64): [1, 2]

In [30]:
X = data['combined_text']
y = data['query']
#label encode it for classification
label = LabelEncoder()
y = label.fit_transform(y)

In [31]:
X_train,y_train,X_test,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [32]:
print("The shape of the X train----------->",X_train.shape)
print("The shape of the y train----------->",y_train.shape)
print("The shape of the X test------------>",X_test.shape)
print("The shape of the y test------------>",y_test.shape)

The shape of the X train-----------> (8608,)
The shape of the y train-----------> (2153,)
The shape of the X test------------> (8608,)
The shape of the y test------------> (2153,)


# tfidf Vectorization